In [63]:
# Query nba.live.endpoints.scoreboard and  list games in localTimeZone
from datetime import datetime, timezone
from dateutil import parser
# from nba_api.live.nba.endpoints import scoreboard
from nba_api.stats.endpoints import scoreboardv2
from nba_api.stats.endpoints.boxscoresummaryv2 import BoxScoreSummaryV2
import pandas as pd
import numpy as np

KEYS_TO_KEEP = {
    'otherStatsDF': ['TEAM_ABBREVIATION', 'TIMES_TIED', 'LARGEST_LEAD'],
    'lineScoreDF': ['TEAM_WINS_LOSSES', 'PTS_QTR1', 'PTS_QTR2', 'PTS_QTR3', 'PTS_QTR4', 'PTS_OT1', 'PTS_OT2', 'PTS_OT3', 'PTS_OT4', 'PTS']
}

f = "{gameId}: {awayTeam} vs. {homeTeam}" 

board = scoreboardv2.ScoreboardV2(day_offset=-1)
print(board.available.get_dict()["data"])
games = [item[0] for item in board.available.get_dict()["data"]]
print(games)

[['0022301164', 1], ['0022301159', 1], ['0022301160', 1], ['0022301163', 1], ['0022301162', 1], ['0022301161', 1], ['0022301165', 1], ['0022301158', 1]]
['0022301164', '0022301159', '0022301160', '0022301163', '0022301162', '0022301161', '0022301165', '0022301158']


In [82]:
### MAIN LABELS ###
NECK_TO_NECK = 'Neck to Neck'
COMEBACK = 'Comeback'
DOWN_TO_THE_WIRE = 'Down to the wire'

#for game in games:
print(games[1])
final_labels = []

box = BoxScoreSummaryV2(games[1])
box_dict = box.get_dict()

0022301159


In [83]:
final_labels = []
# Filtered result sets
otherStats = next(resultSet for resultSet in box_dict['resultSets'] if resultSet['name'] == 'OtherStats')
lineScore = next(resultSet for resultSet in box_dict['resultSets'] if resultSet['name'] == 'LineScore')

# Creating DataFrames
otherStatsDF = pd.DataFrame(otherStats['rowSet'], columns=otherStats['headers'])
lineScoreDF = pd.DataFrame(lineScore['rowSet'], columns=lineScore['headers'])

# Filter for relevant keys
workDF = pd.concat([otherStatsDF[KEYS_TO_KEEP['otherStatsDF']], lineScoreDF[KEYS_TO_KEEP['lineScoreDF']]], axis=1)

####################
#### MAIN LOGIC ####
####################
CLOSE_GAME_LIMIT = 10

quarter_diffs = []
quarter_scores = []
cur_score_A = 0
cur_score_B = 0

### LABEL NECK_TO_NECK ###
# Calculate the difference in scores at the end of each quarter
for i in range(1, 5):
    cur_score_A += workDF.iloc[0][f'PTS_QTR{i}']
    cur_score_B += workDF.iloc[1][f'PTS_QTR{i}']
    quarter_scores.append([cur_score_A, cur_score_B])
    quarter_diffs.append(abs(cur_score_A - cur_score_B))
# If no lead was bigger then Y and end difference smaller then X
if (workDF['LARGEST_LEAD'] < 10).all():
    final_labels.append(NECK_TO_NECK)

### LABEL DOWN TO THE WIRE ###
third_quarter_score = quarter_scores[2]
fourth_quarter_score = quarter_scores[3]
if np.abs(fourth_quarter_score[0] - fourth_quarter_score[1]) < 7:
    final_labels.append(DOWN_TO_THE_WIRE)

### LABEL COMEBACK ###
third_quarter_leader = third_quarter_score.index(max(third_quarter_score))
fourth_quarter_leader = fourth_quarter_score.index(max(fourth_quarter_score))

# If lead of Team A in third quarter is larger then 10 but Team B wins
if np.abs(third_quarter_score[0] - third_quarter_score[1]) > 10 and third_quarter_leader is not fourth_quarter_leader:
    final_labels.append(COMEBACK)

print('-'.join(workDF['TEAM_ABBREVIATION']))
print(final_labels)

CHA-ATL
['Down to the wire', 'Comeback']


In [75]:
workDF.head()

,TEAM_ABBREVIATION,TIMES_TIED,LARGEST_LEAD,TEAM_WINS_LOSSES,PTS_QTR1,PTS_QTR2,PTS_QTR3,PTS_QTR4,PTS_OT1,PTS_OT2,PTS_OT3,PTS_OT4,PTS
0,CHA,7,6,20-60,30,25,22,38,0,0,0,0,115
1,ATL,7,18,36-44,35,26,31,22,0,0,0,0,114


['Close game']
